In [ ]:
# Tools in Langchian allow to LLMs to interact with external data-sources(websites, databases, pdfs). 
# NOTE
'''
    Instead of writing a custom script to scrape a website and preprocess the data, you can let the LLM decide when to scrape and use the data in real time.
    Tools empower agents to dynamically choose which action to take based on input context.
    SUMMARY: 
    Tools are designed to load or fetch only the data that is required, rather than scraping or quering large datasets.
'''


In [2]:
# Tools to search and retrive information from Wikipedia.
from langchain_community.utilities import WikipediaAPIWrapper # helper tool to talk to Wikipedia(pull specific content to directly to the the app.)
from langchain_community.tools import WikipediaQueryRun # takes query and ank Wikipedia for the answer(it uses WikipediaAPIWrapper to find the information).

In [11]:
# Create tool to search from Wikipedia.
wiki_api_wrapper = WikipediaAPIWrapper(top_k_results = 1,doc_content_chars_max = 200) # Configurations to talk with Wikipedia.
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_api_wrapper)

In [12]:
wiki_tool.name

'wikipedia'

In [9]:
# Website as external-data-source - own custom data.
from langchain_community.document_loaders import WebBaseLoader # Scrap web-page
from langchain_text_splitters import RecursiveCharacterTextSplitter # Tokenization
from langchain_community.embeddings import OllamaEmbeddings # Embedding
from langchain_community.vectorstores import FAISS # Vector db(store)

# Load
loader = WebBaseLoader("https://docs.smith.langchain.com/")
web_data = loader.load()
# Transfrom(Tokenization).
web_data = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(web_data)
db = FAISS.from_documents(web_data, OllamaEmbeddings(model="mxbai-embed-large"))
# Create Retriever.
retriever = db.as_retriever()

In [10]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001E019ECBC80>, search_kwargs={})

In [13]:
# Create a tool to do retrival of documents - helps to find right information from collection of data.
from langchain.tools.retriever import create_retriever_tool
# Tool is used to search for langsmith-related information.
web_custom_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)


In [14]:
web_custom_tool.name

'langsmith_search'

In [15]:
# Arxiv-tool, to fetch or load data from Arxiv.
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results = 1, doc_content_chars_max = 200)
arxiv_tool = ArxivQueryRun(api_wrapper = arxiv_wrapper)

In [16]:
arxiv_tool.name

'arxiv'

In [17]:
# Combine all the tools(builtin: Wiki_tool and arxiv_tool, custom_tool: web_custom_tool)
tools = [wiki_tool, arxiv_tool, web_custom_tool]

In [18]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
# Load 'Env-Variables'.
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')

# Initialize LLM-model.
llm = ChatGroq(
    groq_api_key = os.getenv('GROQ_API_KEY'),
    model = "llama-3.1-70b-versatile",
    temperature = 0,
)

In [19]:
# Create Prompts - use builtin prompts.
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")

In [20]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
# Agent: responsible to choose a tool to answer the query.
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [30]:
# Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools,verbose=True)

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith information'}`


